<img width="300px" src="images/learning-tree-logo.svg" alt="Learning Tree logo" />

# Module 3: Exploratory Data Analysis (EDA)

In this module, we cover

- Understanding datasets
- Working wih large datasets
- Creating tidy datasets
- Data processing
- Exploring relationships in the data
- Dimensionality reduction
- Hands-on example of using Python to explore and prepare a dataset

The [notebooks](https://github.com/decisionmechanics/lt539j) for the course are available on GitHub. Clone or download them to follow along.

In this notebook, we make use of the following third-party packages.

```bash
pip install jupyterlab duckdb numpy 'polars[all]' pyjanitor scikit-learn scipy
```

## Understanding datasets

Data comes in many different formats.

- Tabular (e.g. CSV, Excel)
- Semi-structured (e.g. JSON)
- Text
- Images
- Videos
- Audio

When performing data reporting, or ML, we tend to work with tabular data. Even when working with so-called "unstructured" data, such as text, we convert it to a tablular form before analysing it.

In traditional data analysis, we tend to have fields (or variables) that we can group on, summarise, etc.

The fields can be of different types, such as

- integer
- float
- text
- logical
- date
- time
- categorical

Fields should be typed as tightly as possible. If a field should only ever contain dates, don't use a string to represent it.

In tabluar data formats, fields are usually represented by the columns. Typical fields could be country name, continent, year, GDP, etc. 

The rows of a tabular dataset are the observations. Each observation has a value for each of the fields (or a null value, if the data is missing).

Typical observations might be countries, for example.

When building ML models, we categorise the fields into features and targets.

Features, also known as input or independent variables, are used to make the predicitions. They are the attributes of the obversations that we use to predict something of interest.

The target is the something of interest. It's a value, or a class, that we are interested in knowing. Targets might be numeric, such as predicted profit, or categories, such as invest/don't invest.

Targets, when they are given, are used to build supervised learning models.

Note that fields are not fundamentally features or targets. Which group a field falls into depends on the specifics of the modelling activity. And not all fields will be features or targets. Some may be irrelevant to the modelling process (e.g. internal IDs).

Consider the following loan approval dataset.

In [ ]:
import polars as pl

(pl.read_csv("data/loan-approval-dataset.csv"))

We can view the column names.

In [ ]:
(pl.read_csv("data/loan-approval-dataset.csv").columns)

With this dataset, we might want to classify loan applications into approved or rejected (i.e. `loan_status`).

In [ ]:
(pl.read_csv("data/loan-approval-dataset.csv").select("loan_status").unique())

Note than this is a string field, but only has two values. We can convert this to a categorical (or logical) field.

In [ ]:
with pl.StringCache():
    loan_approval_df = pl.read_csv("data/loan-approval-dataset.csv").with_columns(
        pl.col("loan_status").cast(pl.Categorical)
    )

(loan_approval_df.select("loan_status"))

We can split the data into features and target.

In [ ]:
feature_df = loan_approval_df.drop(["loan_id", "loan_status"])

feature_df

In [ ]:
target_df = loan_approval_df.select(["loan_status"])

target_df

Best practice is to _explicitly_ define the fields to be _included_ in the result. So, we might revise the feature selection as follows.

In [ ]:
(
    loan_approval_df.select(
        [
            "no_of_dependents",
            "education",
            "self_employed",
            "income_annum",
            "loan_amount",
            "loan_term",
            "cibil_score",
            "residential_assets_value",
            "commercial_assets_value",
            "luxury_assets_value",
            "bank_asset_value",
        ]
    )
)

More typing, but less opportunity for subtle errors.

## Working with large datasets

Polars allows us to work efficient with large datasets via it's [lazy API](https://docs.pola.rs/user-guide/lazy/).

In [ ]:
(
    pl.scan_csv("data/shark-incidents.csv", infer_schema_length=None)
    .select(["Shark.common.name", "Victim.injury"])
    .filter(~pl.all_horizontal(pl.all().is_null()))
)

The lazy API allows us to build up a query plan which can then be optimized before attempting to load the data.

In [ ]:
(
    pl.scan_csv("data/shark-incidents.csv", infer_schema_length=None)
    .select(["Shark.common.name", "Victim.injury"])
    .filter(~pl.all_horizontal(pl.all().is_null()))
    .show_graph()
)

To execute the plan we can `collect` the data.

In [ ]:
(
    pl.scan_csv("data/shark-incidents.csv", infer_schema_length=None)
    .select(["Shark.common.name", "Victim.injury"])
    .filter(~pl.all_horizontal(pl.all().is_null()))
    .collect()
)

For small datasets, it's convenient to use eager methods, such as `read_csv`. But, when your dataset is large, and you don't need all the data, using the lazy API can significantly improve performance.

Another performance improvement is to use more efficient file formats. Parquet files, for example, store data by column. This makes it more efficient when you only want certain columns.

Parquet files also store metadata, such as the column types (e.g. float, string), making them more suited to data analysis work.

If you are going to be loading a CSV/Excel data file multiple times, it often makes sense to convert it to parquet format.

In [ ]:
(
    pl.scan_csv("data/shark-incidents.csv", infer_schema_length=None)
    .filter(~pl.all_horizontal(pl.all().is_null()))
    .collect()
    .write_parquet("temp/shark-incidents.parquet")
)

In [ ]:
(
    pl.scan_parquet("temp/shark-incidents.parquet")
    .select(["Shark.common.name", "Victim.injury"])
    .collect()
)

We can see that, when you only need a subset of the data, parquet files can be much faster.

In [ ]:
%%timeit

(
    pl.scan_parquet("temp/shark-incidents.parquet")
    .select(["Shark.common.name", "Victim.injury"])
    .collect()
)

In [ ]:
%%timeit

(
    pl.scan_csv("temp/shark-incidents.csv", infer_schema_length=None)
    .select(["Shark.common.name", "Victim.injury"])
    .collect()
)

[DuckDB](https://duckdb.org) is another efficient format for working with large datasets.

In [ ]:
import os

import duckdb

DB_FILE_PATH = "temp/shark-incidents.db"

if os.path.isfile(DB_FILE_PATH):
    os.remove(DB_FILE_PATH)

shark_incidents_df = (
    pl.scan_csv("temp/shark-incidents.csv", infer_schema_length=None)
    .rename(
        {
            "Shark.common.name": "shark_common_name",
            "Victim.injury": "victim_injury",
        }
    )
    .select(
        [
            "shark_common_name",
            "victim_injury",
        ]
    )
    .collect()
)


database = duckdb.connect(DB_FILE_PATH)

try:
    database.sql("CREATE TABLE incidents AS SELECT * FROM shark_incidents_df")
finally:
    database.close()

## Creating tidy datasets

When first encountering a dataset, it's important to have a quick look at the raw data. If it's a text format (e.g. CSV) you can look at it using CLI tools or open it in a text editor (such as Visual Studio Code).

Consider the national GDP file downloaded in CSV format from the World Bank's website.

In [ ]:
!head -n 5 data/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_31795.csv

We can see that there's some preamble here that might cause a problem if we naively try to load it as a well-formed CSV file.

In [ ]:
(
    pl.read_csv(
        "data/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_31795.csv", truncate_ragged_lines=True
    )
)

Datasets often have problems at the end as well.

In [ ]:
!tail -n 5 data/shark-incidents.csv

We can also investigate lines in the middle of the file.

In [ ]:
line_number = 20

with open("data/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_31795.csv") as f:
    for index, line in enumerate(f):
        if index == line_number - 1:
            print(line)

            break;

Or, if you prefer to use CLI tools, use the following.

In [ ]:
!awk 'NR==20' data/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_31795.csv

Note the trailing comma on the end of the line.

Just getting the data into a useful tabular format can involve a significant amount of custom coding.

Let's get the World Bank GDP data into a form we can work with.

In [ ]:
gdp_df = pl.read_csv("data/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_31795.csv", skip_rows=4)

gdp_df

When a data frame is read, it makes sense to normalise the names. The `pyjanitor` package can do this for us.

In [ ]:
import janitor.polars

gdp_df = pl.read_csv(
    "data/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_31795.csv", skip_rows=4
).clean_names()

gdp_df

It's now in a form where we have access to the correct values. But it's still not ideal.

First thing to note is that we have a spurious column at the end. We can remove that.

In [ ]:
gdp_df = (
    pl.read_csv("data/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_31795.csv", skip_rows=4)
    .clean_names()
    .drop("")
)

gdp_df

This dataset isn't in [tidy format](https://tidyr.tidyverse.org/articles/tidy-data.html).

> [In a tidy dataset,] every value belongs to a variable and an observation. A variable contains all values that measure the same underlying attribute (like height, temperature, duration) across units. An observation contains all values measured on the same unit (like a person, or a day, or a race) across attributes.

In the World Bank dataset, the year is a field, but it's defined across multiple columns. This makes is harder to work with, and results in lots of blank "cells" due to missing values.

We can unpivot the "worksheet" format to a tidy format.

In [ ]:
(
    gdp_df.unpivot(
        index=["country_name", "country_code", "indicator_name", "indicator_code"],
        variable_name="year",
        value_name="gdp",
    )
)

This is cleaner, but there are now a lot of missing ("") values. We can replace those with nulls.

In [ ]:
(
    gdp_df.unpivot(
        index=["country_name", "country_code", "indicator_name", "indicator_code"],
        variable_name="year",
        value_name="gdp",
    ).with_columns(
        pl.col("year").replace("", None),
        pl.col("gdp").replace("", None),
    )
)

We can see that every field is using the string data type. `year` should really be an integer and `gdp` a float.

In [ ]:
(
    gdp_df.unpivot(
        index=["country_name", "country_code", "indicator_name", "indicator_code"],
        variable_name="year",
        value_name="gdp",
    )
    .with_columns(
        pl.col("year").replace("", None),
        pl.col("gdp").replace("", None),
    )
    .with_columns(
        pl.col("year").cast(pl.UInt16),
        pl.col("gdp").cast(pl.Float64),
    )
)

Now what we've cleaned the data up, we can bring it all together and store it as a parquet file.

In [ ]:
(
    pl.read_csv("data/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_31795.csv", skip_rows=4)
    .clean_names()
    .unpivot(
        index=["country_name", "country_code", "indicator_name", "indicator_code"],
        variable_name="year",
        value_name="gdp",
    )
    .with_columns(
        pl.col("year").replace("", None),
        pl.col("gdp").replace("", None),
    )
    .with_columns(
        pl.col("year").cast(pl.UInt16),
        pl.col("gdp").cast(pl.Float64),
    )
    .write_parquet("temp/world-bank-gdp.parquet")
)

Read it back in to check.

In [ ]:
pl.read_parquet("temp/world-bank-gdp.parquet")

Note that it might have made sense to filter out observations with missing values, convert some of the string fields to categoricals and remove fields with single values before saving as a parquet file. We will look at performing these operations in the next section.

## Data processing

Load the World Bank's GDP data.

In [ ]:
gdp_df = pl.read_parquet("temp/world-bank-gdp.parquet")

gdp_df

### Handling missing values

Clearly, there's some missing GDP values (and years). How many values are missing?

In [ ]:
gdp_df.null_count()

Let's drop the observations that don't have a GDP value.

In [ ]:
gdp_df = pl.read_parquet("temp/world-bank-gdp.parquet").filter(
    pl.col("gdp").is_not_null(),
)

gdp_df

Any missing values now?

In [ ]:
gdp_df.null_count()

### Encoding categorical variables

The string fields in this dataset could be considered categoricals. Let's change their type.

In [ ]:
import polars.selectors as cs

gdp_df = (
    pl.read_parquet("temp/world-bank-gdp.parquet")
    .filter(
        pl.col("gdp").is_not_null(),
    )
    .with_columns(
        (~cs.numeric()).cast(pl.Categorical),
    )
)

gdp_df

In [ ]:
gdp_df

Is there more than one indicator in this dataset?

In [ ]:
(gdp_df.select(["indicator_name", "indicator_code"]).unique())

As these columns don't add anything, we can drop them.

In [ ]:
gdp_df = (
    pl.read_parquet("temp/world-bank-gdp.parquet")
    .filter(
        pl.col("gdp").is_not_null(),
    )
    .with_columns(
        (~cs.numeric()).cast(pl.Categorical),
    )
    .drop(["indicator_name", "indicator_code"])
)

gdp_df

Are there any duplicated observations?

In [ ]:
(gdp_df.drop("gdp").filter(gdp_df.is_duplicated()))

When working with categorical data, we sometimes want to convert it to multiple columns that signify the presence or absence of a given _value_.

This makes it easy for ML algorithms to include the information conveyed by the categorical field.

This approach is known as onehot encoding. And the new present/absent fields are know as dummy variables.

In [ ]:
shark_incident_df = pl.read_parquet("data/shark-incidents.parquet")

(
    shark_incident_df.select("state")
    .with_columns(pl.col("state").alias("temporary_state"))
    .to_dummies("state", drop_first=True)
)

In [ ]:
(shark_incident_df.select(cs.float()))

In [ ]:
shark_incident_df

### Scaling

When data is at very different scales, it can confuse some ML algorithms---or at least make them less efficient.

In [ ]:
weight_height_df = pl.DataFrame(
    {
        "weight_g": [80_000, 81_000, 82_000, 83_000],
        "height_m": [1.87, 1.86, 1.57, 1.56],
    }
)

In [ ]:
(
    weight_height_df.plot.point(
        x="weight_g",
        y="height_m",
    )
)

Plotting data automatically scales it. Points 1 and 2 appear to be close, although they are over 1000 units apart.

We can normalise the fields by transforming them to z-scores---values with a mean of 0 and a standard deviation of 1.

In [ ]:
from scipy.stats import zscore

weight_height_z_score_df = pl.from_numpy(
    zscore(weight_height_df.to_numpy(), ddof=1)
).pipe(lambda df_: df_.rename(dict(zip(df_.columns, weight_height_df.columns))))

weight_height_z_score_df

In [ ]:
weight_height_z_score_df.mean()

In [ ]:
weight_height_z_score_df.std()

Note that this doesn't change the _relationships_ in the dataset---just the values.

In [ ]:
(
    weight_height_z_score_df.plot.point(
        x="weight_g",
        y="height_m",
    )
)

## Exploring relationships in the data

Understanding a dataset involves exploring the relationships in the data. Key relationships include

- Relationships between the values in a field (e.g. their distribution)
- Relationships between fields (correlations)

Consider the World Bank's GDP dataset.

In [ ]:
gdp_df = pl.read_csv("data/world-bank-gdp.csv")

gdp_df

We can look at how the UK's GDP has changed over the years.

In [ ]:
import altair as alt

(
    gdp_df.filter(
        pl.col("country_code") == "GBR",
    )
    .plot.line(x="year", y="gdp")
    .properties(width=500, height=200)
)

We can also look at the distribution of GDP/capita for 2023.

Note that the following code should work, but there's a bug at the time of writing.

```python
(
    gdp_df.with_columns((pl.col("gdp") / pl.col("population")).alias("gdp_per_capita"))
    .filter(
        pl.col("year") == 2023,
    )
    .plot.boxplot(x="gdp")
    .properties(width=500, height=200)
)
```

In [ ]:
import altair as alt

alt.Chart(
    gdp_df.with_columns(
        (pl.col("gdp") / pl.col("population")).alias("gdp_per_capita")
    ).filter(
        pl.col("year") == 2023,
    )
).mark_boxplot().encode(alt.X("gdp_per_capita")).properties(width=500, height=200)

Or the correlation between population and GDP.

In [ ]:
(
    gdp_df.filter(
        pl.col("year") == 2023,
    )
    .plot.point(x="population", y="gdp")
    .properties(width=500, height=200)
)

In [ ]:
gdp_df.select(pl.corr("population", "gdp"))

Let's look at some data from a [research paper](https://www.pnas.org/doi/abs/10.1073/pnas.1209746109). This paper was retracted in 2021.

We'll examine the dataset underlying the conclusions and see why there were [serious concerns](https://datacolada.org/98) about the research.

The research claimed to show that having people signing a statement of honest intent _prior_ to completing a form made their disclosures more truthful than if you asked them to sign it at the _end_ of the form.

They choose to demonstrate this using car insurance data, where drivers were required to submit odometer readings.

The original data is in an Excel workbook (DrivingdataAll (1).xls). We'll use a cleaned up version (e.g. cleaner column names).

In [ ]:
dishonesty_study_df = pl.read_parquet("data/dishonesty-study.parquet")

dishonesty_study_df

Customers can have up to four cars. Baseline and updated odometer read are provided for each car. The baseline readings were earlier readings supplied to the company.

Let's consider the miles driven in car 1 during the period between the baseline reading and the updated reading. We have data on how many miles people drive per year.

![Annual mileage per car](https://www.bymiles.co.uk/wp-content/uploads/2023/12/MOT-Data-2024-Annual-mileage-car@3x-1-1000x669.png)

Some people don't drive at all, a lot of people drive a little, and a few drive a lot.

Here's the distribution from the data in the paper.

In [ ]:
(
    dishonesty_study_df.with_columns(
        (pl.col("update_car_1") - pl.col("baseline_car_1")).alias("miles_driven_car_1")
    )
    .get_column("miles_driven_car_1")
    .hist(bin_count=50)
    .plot.bar(
        x="breakpoint",
        y="count",
    )
    .properties(width=500)
)

That's pretty much a uniform distribution that runs from 0 to 50,000.

The distributions for the other cars are similarly problematic.

In [ ]:
(
    dishonesty_study_df.with_columns(
        (pl.col("update_car_4") - pl.col("baseline_car_4")).alias("miles_driven_car_4")
    )
    .get_column("miles_driven_car_4")
    .hist(bin_count=50)
    .plot.bar(
        x="breakpoint",
        y="count",
    )
    .properties(width=500)
)

When people estimate car mileages they tend to round them. We can see this in the baseline data for car 1.

In [ ]:
(
    dishonesty_study_df.with_columns(
        (pl.col("baseline_car_1") % 1000).alias("baseline_car_1_rounding_3")
    )
    .get_column("baseline_car_1_rounding_3")
    .value_counts()
    .with_columns((pl.col("count") / pl.sum("count")).alias("proportion"))
    .plot.bar(
        x="baseline_car_1_rounding_3",
        y=alt.Y("proportion", scale=alt.Scale(domain=[0, 0.12])),
    )
    .properties(width=800)
)

However, when we look at the updated data for car one, we see no evidence of rounding.

In [ ]:
(
    dishonesty_study_df.with_columns(
        (pl.col("update_car_1") % 1000).alias("update_car_1_rounding_3")
    )
    .get_column("update_car_1_rounding_3")
    .value_counts()
    .with_columns((pl.col("count") / pl.sum("count")).alias("proportion"))
    .plot.bar(
        x="update_car_1_rounding_3",
        y=alt.Y("proportion", scale=alt.Scale(domain=[0, 0.12])),
    )
    .properties(width=800)
)

How about looking at the frequency of the last digit?

In [ ]:
(
    dishonesty_study_df.with_columns(
        (pl.col("baseline_car_1") % 10).alias("baseline_car_1_rounding_1")
    )
    .get_column("baseline_car_1_rounding_1")
    .value_counts()
    .with_columns((pl.col("count") / pl.sum("count")).alias("proportion"))
    .plot.bar(
        x="baseline_car_1_rounding_1",
        y=alt.Y("proportion", scale=alt.Scale(domain=[0, 0.25])),
    )
    .properties(width=500)
)

Again, the updated data doesn't look right.

In [ ]:
(
    dishonesty_study_df.with_columns(
        (pl.col("update_car_1") % 10).alias("update_car_1_rounding_1")
    )
    .get_column("update_car_1_rounding_1")
    .value_counts()
    .with_columns((pl.col("count") / pl.sum("count")).alias("proportion"))
    .plot.bar(
        x="update_car_1_rounding_1",
        y=alt.Y("proportion", scale=alt.Scale(domain=[0, 0.25])),
    )
    .properties(width=500)
)

There was an intriguing formatting issue in the worksheet. _Exactly_ half of the values in the the `baseline_car_1` column were displayed using the Cambria font. The other half used Calibri.

In [ ]:
(dishonesty_study_df.filter(pl.col("font") == "Cambria").select(pl.len()))

In [ ]:
(dishonesty_study_df.filter(pl.col("font") == "Calibri").select(pl.len()))

We can split the dataset by font, with each font field sorted by `baseline_car_1`.

In [ ]:
font_df = pl.DataFrame(
    {
        "cambria": dishonesty_study_df.filter(pl.col("font") == "Cambria")
        .sort("baseline_car_1")
        .get_column("baseline_car_1"),
        "calibri": dishonesty_study_df.filter(pl.col("font") == "Calibri")
        .sort("baseline_car_1")
        .get_column("baseline_car_1"),
    }
).with_columns((pl.col("cambria") - pl.col("calibri")).alias("difference"))

font_df

Is there a correlation between these columns?

In [ ]:
font_df.select(pl.corr("cambria", "calibri"))

How is the difference between the two columns distributed?

In [ ]:
(
    font_df.get_column("difference")
    .hist(bin_count=50)
    .plot.bar(x="breakpoint", y="count")
    .properties(width=500)
)

Could someone have copied all the data and added a random number with some distribution running between 0 and 1000?

## Dimensionality reduction

High dimensional data (i.e. many features) can result in performance and accuracy problems in ML. This is often referred to as the curse of dimensionality.

There are techniques that allow us to to reduce the dimensionality of the data, while minimising the loss of information.

Consider the following (meaningless) dataset with two features in it.

In [ ]:
import numpy as np

original_data = np.stack(
    [
        [
            0.72366164,
            1.44194784,
            2.20955984,
            2.89736675,
            3.43509835,
            4.17267258,
            4.91907454,
            5.70487071,
            6.40549306,
            6.93480835,
        ],
        [
            0.69055192,
            1.38647928,
            2.03308085,
            2.7594875,
            3.63596946,
            4.31260879,
            4.9804204,
            5.60883779,
            6.322429,
            7.20732728,
        ],
    ]
)

Here's how it looks when plotted.

In [ ]:
import matplotlib.pyplot as plt

plt.xlim(-1, 11)
plt.ylim(-1, 11)

plt.scatter(x=original_data[0], y=original_data[1]);

If we rotate this data 45&deg;, we reposition it with respect to the axes, but retain all the relationships between the points.

In [ ]:
def rotate_points(points, theta):
    theta_radians = theta * np.pi / 180

    rotation = [
        [np.cos(theta_radians), -np.sin(theta_radians)],
        [np.sin(theta_radians), np.cos(theta_radians)],
    ]

    return rotation @ points


transformed_data = rotate_points(original_data, -45)

plt.xlim(-1, 11)
plt.ylim(-1, 11)

plt.scatter(x=original_data[0], y=original_data[1], color="#ccc")
plt.scatter(x=transformed_data[0], y=transformed_data[1]);

Note how the majority of the variation in this data is across the x-axis. There's very little variation in across the y-axis.

If we set the y values to 0, we now have a one-dimensional (single feature) dataset that retains the bulk of the variation.

In [ ]:
plt.scatter(x=transformed_data[0], y=np.zeros(10));

### Principal Component Analysis

Principal Component Analysis (PCA) is a popular dimensionality reduction approach.

We'll use it here to reduce the dimensionality of the [Palmer penguins dataset](https://allisonhorst.github.io/palmerpenguins/articles/intro.html).

In [ ]:
penguin_df = pl.read_csv("data/penguins.csv").drop_nulls()

penguin_df

PCA can’t handle missing values, so we’ll just remove observations with missing data. 

Probabilistic PCA (PPCA) can be used when you have missing data.

PCA only works with numeric values.

In [ ]:
penguin_numeric_df = penguin_df.select(cs.numeric().exclude("year"))

penguin_numeric_df

PCA is affected by scale, so we'll convert our dataset to z-scores.

In [ ]:
penguin_z_score_df = pl.from_numpy(zscore(penguin_numeric_df.to_numpy(), ddof=1)).pipe(
    lambda df_: df_.rename(dict(zip(df_.columns, penguin_numeric_df.columns)))
)

penguin_z_score_df

Transform the dataset to its principal components.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()

principal_components = pca.fit_transform(penguin_z_score_df.to_numpy())

principal_component_df = pl.concat(
    [
        penguin_df.select("species"),
        pl.from_numpy(
            principal_components,
            ["component1", "component2", "component3", "component4"],
        ),
    ],
    how="horizontal",
)

principal_component_df

How much of the variable is explained by each principal component?

In [ ]:
np.cumsum(pca.explained_variance_ratio_)

We can see that 88% of the total variance is explained by the first two components.

This lets us visualise the data as a two-dimensional scatterplot.

In [ ]:
(
    principal_component_df.plot.point(
        x="component1", y="component2", color="species"
    ).properties(width=500, height=500)
)

We can see that the Gentoos can be trivially identified, but seperating the Adelies from the Chinstraps is more challenging.

## Hands-on example of using Python to explore and prepare a dataset

Explore a [movie dataset](https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset) from Kaggle.

There are two CSV files.

- `movies.csv` which contains data about the movies
- `movie_ratings.csv` which contains ratings for the movies

Check the top and bottom of the file.

In [ ]:
!head -n 3 data/movies.csv

In [ ]:
!tail -n 3 data/movies.csv

Looks like a valid CSV file. Load it into a dataframe.

In [ ]:
movie_df = pl.read_csv("data/movies.csv")

movie_df

Get a summary of the schema.

In [ ]:
movie_df.glimpse()

The column names look clean, so there's no need to transform them.

The data type of the `release_date` field doesn't seem to have been inferred correctly. Convert it to a date.

In [ ]:
movie_df = pl.read_csv("data/movies.csv").with_columns(
    pl.col("release_date").str.to_date()
)

movie_df.glimpse()

Is there any missing data?

In [ ]:
movie_df.null_count()

Yes, but probably not in the important fields. Seems pretty clean.

Examine the revenue distribution.

In [ ]:
(movie_df.get_column("revenue").hist(bin_count=50).plot.bar(x="breakpoint", y="count"))

Quite a lot of movies with low revenue.

In [ ]:
(movie_df.sort("revenue").limit(5))

Movies with $0 revenue (and $0 budget) seem unlikely. Some data issues there. 

In [ ]:
(
    movie_df.filter(
        pl.col("revenue") > 0,
    )
    .get_column("revenue")
    .hist(bin_count=50)
    .plot.bar(x="breakpoint", y="count")
)

That looks more reasonable. What about movies that made less than $500m?

In [ ]:
(
    movie_df.filter(
        pl.col("revenue") > 0,
        pl.col("revenue") < 500_000_000,
    )
    .get_column("revenue")
    .hist(bin_count=50)
    .plot.bar(x="breakpoint", y="count")
)

How is `budget` distributed?

In [ ]:
(
    movie_df.filter(
        pl.col("budget") > 0,
    )
    .get_column("budget")
    .hist(bin_count=50)
    .plot.bar(x="breakpoint", y="count")
)

Is there a relationship between budget and revenue?

In [ ]:
movie_sample_df = movie_df.filter(
    pl.col("budget") > 0,
    pl.col("revenue") > 0,
).sample(5000)

In [ ]:
(movie_sample_df.plot.point(x="budget", y="revenue"))

In [ ]:
(
    movie_df.filter(
        pl.col("budget") > 0,
        pl.col("revenue") > 0,
    ).select(
        pl.corr("budget", "revenue"),
    )
)

What period is covered by the dataset?

In [ ]:
(
    movie_df.select(
        [
            pl.min("release_date").alias("min_release_date"),
            pl.max("release_date").alias("max_release_date"),
        ]
    )
)

How is `runtime` distributed?

In [ ]:
(
    movie_df.filter(
        pl.col("runtime") > 0,
        pl.col("runtime") < 240,
    )
    .get_column("runtime")
    .hist(bin_count=50)
    .plot.bar(x="breakpoint", y="count")
)

How does it look if you remove the filter?

In [ ]:
(movie_df.get_column("runtime").hist(bin_count=50).plot.bar(x="breakpoint", y="count"))

Explore the rating data.

In [ ]:
movie_rating_df = pl.read_csv("data/movie-ratings-sample.csv").rename(
    {
        "userId": "user_id",
        "movieId": "movie_id",
    }
)

movie_rating_df

View the distribution of the ratings.

In [ ]:
(movie_rating_df.get_column("rating").value_counts().plot.bar(x="rating", y="count"))

Does this seem OK?

What's the average rating for horror movies?

In [ ]:
(
    movie_df.filter(pl.col("genres").str.contains("Horror"))
    .join(movie_rating_df, left_on="id", right_on="movie_id")
    .select(pl.col("rating").mean())
)

How does that compare to mystery movies?

In [ ]:
(
    movie_df.filter(pl.col("genres").str.contains("Mystery"))
    .join(movie_rating_df, left_on="id", right_on="movie_id")
    .select(pl.col("rating").mean())
)

Create a dataset containing the all the Science Fiction movies and their reviews. Only include fields you think are interesting.

Save your dataset as a parquet file.

In [ ]:
(
    movie_df.filter(pl.col("genres").str.contains("Science Fiction"))
    .join(movie_rating_df, left_on="id", right_on="movie_id")
    .select(
        [
            "original_title",
            "genres",
            "release_date",
            "runtime",
            "budget",
            "revenue",
            "popularity",
            "user_id",
            "rating",
        ]
    )
    .write_parquet("temp/horror_movie_ratings.parquet")
)

### Explore a job change dataset

Kaggle has a [dataset](https://www.kaggle.com/datasets/arashnic/hr-analytics-job-change-of-data-scientists?datasetId=1019790&sortBy=voteCount&select=aug_train.csv) that documents whether data scientists are looking to change jobs.

It has the following features.

- `enrollee_id` : Unique ID for candidate.
- `city`: City code.
- `city_development_index` : Developement index of the city (scaled).
- `gender`: Gender of candidate
- `relevent_experience`: Relevant experience of candidate
- `enrolled_university`: Type of University course enrolled if any
- `education_level`: Education level of candidate
- `major_discipline` :Education major discipline of candidate
- `experience:` Candidate total experience in years
- `company_size`: No of employees in current employer's company
- `company_type` : Type of current employer
- `last_new_job`: Difference in years between previous job and current job
- `training_hours`: training hours completed
- `target`: 0 – Not looking for job change, 1 – Looking for a job change

In [ ]:
job_change_df = pl.read_csv("data/data-scientist-job-change.csv")

Explore and clean this dataset.

Some things you might consider are

- Missing data
- Distribution of individual features
- Relationships between features

For example, look at how education level interacts with interest in changing job.

We create a custom ordering on `education_level` to sort the chart.

In [ ]:
with pl.StringCache():
    education_level = ["Primary School", "High School", "Graduate", "Masters", "Phd"]

    pl.Series(education_level).cast(pl.Categorical)
    
    df_ = job_change_df.with_columns(
        pl.col("education_level").cast(pl.Categorical)
    )
    
(
    df_
    .filter(
        pl.col("education_level").is_not_null(),
    )
    .group_by(["education_level", "target"])
    .agg(
        pl.len().alias("count"),
    )
    .sort(
        pl.col("education_level").to_physical()        
    )
    .plot.bar(
        x = alt.X("sum(count)", title=None),
        y = alt.Y("education_level", sort=None),
        color="target",
    )
    .properties(
        width=500,
        height=200,
    )
)

## Hands-on exploration of OKCupid data

Open the OKCupid dataset and explore it. Most of the fields are categorical.

How clean is the data?

Can you identify any correlations?

In [ ]:
ok_cupid_df = pl.read_csv("data/ok-cupid.csv", infer_schema_length=None)

ok_cupid_df.head()